In [3]:
import pandas as pd
import numpy as np
import inflection
import plotly.express as px
import folium
from folium.plugins import MarkerCluster

df1 = pd.read_csv('zomato.csv')
df = df1.copy()

df = df.drop_duplicates(subset=['Restaurant ID', 'Restaurant Name', 'Address', 'Votes'])

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df
df = rename_columns(df)


df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0] if isinstance(x, str) else x)

#Retirando valores zerados da coluna average_cost_for_two
df = df.loc[df['average_cost_for_two'] != 0, :]
#Retirando valores nulos da coluna cuisines
df.dropna( subset=['cuisines'], inplace=True)
df = df.reset_index(drop=True)

#Contagem de dados nulos
# df.isnull().sum()


In [4]:
# Atribuir cores aos ratings
COLORS = {
 "3F7E00": "darkgreen",
 "5BA829": "green",
 "9ACD32": "lightgreen",
 "CDD614": "orange",
 "FFBA00": "red",
 "CBCBC8": "darkred",
 "FF7800": "darkred",
 }
def color_name(color_code):
    return COLORS[color_code]

color_name_vectorized = np.vectorize(color_name)
df['color_name'] = color_name_vectorized(df['rating_color'])


In [5]:
#Coluna com o nome dos países
COUNTRIES = {
 1: "India",
 14: "Australia",
 30: "Brazil",
 37: "Canada",
 94: "Indonesia",
 148: "New Zeland",
 162: "Philippines",
 166: "Qatar",
 184: "Singapure",
 189: "South Africa",
 191: "Sri Lanka",
 208: "Turkey",
 214: "United Arab Emirates",
 215: "England",
 216: "United States of America",
 }
def country_name(country_id):
    return COUNTRIES[country_id]

country_name_vectorized = np.vectorize(country_name)
df['country_name'] = country_name_vectorized(df['country_code'])



In [6]:
#Determinaçao do tipo de preço
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

create_price_tye = np.vectorize(create_price_tye)
df['price_type'] = create_price_tye(df['price_range'])



In [7]:
#Correction of a wrong value in the average cost for two column
df.loc[350, 'average_cost_for_two'] = 120

#Adição do dolar australiano e cambio de moedas para dolar
df.loc[df['country_code'] == 14, 'currency'] = 'Australian Dollar'
taxas_cambio = {
    "Dollar($)": 1.00,
    "Botswana Pula(P)": 13.38, 
    "Brazilian Real(R$)": 5.56,
    "Emirati Diram(AED)": 3.67, 
    "Indian Rupees(Rs.)": 85.79,
    "Indonesian Rupiah(IDR)": 16258.05,
    "NewZealand($)":1.65,
    "Pounds(£)": 0.74,
    "Qatari Rial(QR)": 3.64,
    "Rand(R)": 17.78,
    "Sri Lankan Rupee(LKR)": 299.17,
    "Turkish Lira(TL)": 39.28,
    "Australian Dollar": 1.54    
}

def converter_para_dolar(valor, moeda_origem):
    if moeda_origem in taxas_cambio:
        taxa_cambio = taxas_cambio[moeda_origem]
        valor_em_dolar = valor / taxa_cambio
        return valor_em_dolar
    else:
        return None
    
df['average_cost_dolar'] = df.apply(lambda linha: converter_para_dolar(linha['average_cost_for_two'], linha['currency']), axis=1)
df['average_cost_dolar'] = df['average_cost_dolar'].round(2)

In [8]:
df


,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,color_name,country_name,price_type,average_cost_dolar
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,3,4.6,3F7E00,Excellent,619,darkgreen,Philippines,expensive,82.21
1,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,4,4.7,3F7E00,Excellent,469,darkgreen,Philippines,gourmet,231.69
2,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,0,3,4.4,5BA829,Very Good,867,green,Philippines,expensive,59.79
3,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,3,4.4,5BA829,Very Good,858,green,Philippines,expensive,52.32
4,6304833,Manam,162,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,...,0,3,4.7,3F7E00,Excellent,930,darkgreen,Philippines,expensive,52.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6874,5912546,Eataly,208,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.017326,41.065322,Italian,...,0,4,4.3,5BA829,Very Good,1367,green,Turkey,gourmet,7.64
6875,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,208,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052620,41.050280,Fast Food,...,0,2,4.5,3F7E00,Excellent,1172,darkgreen,Turkey,normal,1.15
6876,5923535,Boon Cafe & Restaurant,208,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052623,41.050717,Restaurant Cafe,...,0,4,4.2,5BA829,Very Good,1160,green,Turkey,gourmet,3.56
6877,5914190,Kanaat Lokantası,208,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.016590,41.025741,Home-made,...,0,3,4.0,5BA829,Very Good,770,green,Turkey,expensive,2.42


# Geral

In [80]:
# Quantos restaurantes unicos estao registrados?
unique_restaurants = df['restaurant_id'].nunique()
print(f"Number of unique restaurants: {unique_restaurants}")

Number of unique restaurants: 6879


In [40]:
# Quantos paises diferentes estao registrados?
unique_countries = df['country_name'].nunique()
print(f"Number of unique countries: {unique_countries}")

Number of unique countries: 15


In [41]:
# Quantas cidades diferentes estao registradas?
unique_cities = df['city'].nunique()
print(f"Number of unique cities: {unique_cities}")

Number of unique cities: 125


In [42]:
# Qual o total de avaliacoes?
total_votes = df['votes'].sum()
print(f"Total votes: {total_votes}")

Total votes: 4188512


In [43]:
# Total of cuisines registered?
total_cuisines = df['cuisines'].nunique()
print(f"Total unique cuisines: {total_cuisines}")

Total unique cuisines: 164


# 2 -Countries profile

In [ ]:
# Which country has more cities registered?

country_city_counts = df.loc[:, ['country_name', 'city']].groupby('country_name').agg({'city': 'nunique'}).reset_index()
country_city_counts = country_city_counts.sort_values(by='city', ascending=False).reset_index(drop=True)
# print("Countries with the most cities registered:")
# print(country_city_counts)

fig = px.bar(
    country_city_counts,
    x='country_name',
    y='city',
    orientation='v',
    title='Number of Cities registered per Country',
    color='country_name',
    text='city',
)
fig.update_traces(textposition='auto')

fig.show()

In [ ]:
# Paleta de cores bem distintas, vibrantes e sem preto

COUNTRY_COLORS = {
    "India": "#719BF7",                
    "Australia": "#E68435",            
    "Brazil": "#F1E209",               
    "Canada": "#6B4AFF",               
    "Indonesia": "#7389CF",            
    "New Zeland": "#00F3FB",           
    "Philippines": "#BB3C3C",          
    "Qatar": "#F5ACC4",                
    "Singapure": "#4C6CFD",            
    "South Africa": "#F4511E",         
    "Sri Lanka": "#81D376",            
    "Turkey": "#E0E1E9",               
    "United Arab Emirates": "#28EE0E", 
    "England": "#EB6DBB",              
    "United States of America": "#0059BF" 
}

def get_country_palette(country_list):
    return [COUNTRY_COLORS[country] for country in country_list]

# Exemplo de uso:
country_city_counts = country_city_counts.sort_values(by='city', ascending=False).reset_index(drop=True)
palette = get_country_palette(country_city_counts['country_name'])

fig = px.bar(
    country_city_counts,
    x='country_name',
    y='city',
    orientation='v',
    title='Number of Cities registered per Country',
    color='country_name',
    text='city',
    color_discrete_sequence=palette
)
fig.update_layout(showlegend=False)
fig.update_traces(textposition='auto')
fig.show()

In [ ]:
# Which country has the most restaurants?
most_restaurants_count = df.loc[:, ['country_name', 'restaurant_id']].groupby('country_name').agg({'restaurant_id': 'nunique'}).reset_index()
most_restaurants_count = most_restaurants_count.sort_values(by='restaurant_id', ascending=False).reset_index(drop=True)
# print("Countries with the most restaurants registered:")    
# print(most_restaurants_count)

fig = px.bar(
    most_restaurants_count,
    x='country_name',
    y='restaurant_id',
    title='Quantity of restaurants registered in each country',
    text='restaurant_id',  # Mostra os valores sobre as barras
    color='country_name'
)
fig.update_traces(textposition='auto')  # Posição do texto automatica nas barras
fig.show()


In [ ]:
# Which country has the most restaurants with price_range = 4 ?
country_expensive_restaurants = df.loc[df['price_range'] == 4, ['country_name', 'restaurant_id']].groupby('country_name').agg({'restaurant_id': 'nunique'}).reset_index()
country_expensive_restaurants = country_expensive_restaurants.sort_values(by='restaurant_id', ascending=False).reset_index(drop=True)
print("Countries with the most gourmet restaurants registered:")    
print(country_expensive_restaurants)

In [ ]:
# Which country has the most variety of cuisines?
cuisines_variety = df.loc[:, ['country_name','cuisines']].groupby('country_name').agg({'cuisines': 'nunique'})
cuisines_variety = cuisines_variety.sort_values(by='cuisines', ascending=False).reset_index()
cuisines_variety

In [ ]:
#Which country has the most reviews?
most_reviews_country = df.loc[:,['country_name', 'votes']].groupby('country_name').sum()
most_reviews_country = most_reviews_country.sort_values(by='votes', ascending=False)
most_reviews_country


In [ ]:
#Which country has the most restaurants that delivers?
country_delivers = df.loc[df['has_online_delivery'] == 1, ['country_name', 'restaurant_id']].groupby('country_name').count()
country_delivers = country_delivers.sort_values(by='restaurant_id', ascending=False).reset_index()
country_delivers

In [ ]:
#Which country has the most restaurants which accept reservations?
reservations_country = df.loc[df['has_table_booking'] == 1, ['country_name', 'restaurant_id']].groupby('country_name').count()
reservations_country = reservations_country.sort_values(by='restaurant_id',ascending=False).reset_index()
reservations_country

In [ ]:
#Qual o nome do país que possui, na média, a maior quantidade de avaliações  registrada?
avg_rating_country = df.groupby('country_name')['votes'].mean()
avg_rating_country = avg_rating_country.sort_values(ascending=False)
avg_rating_country

In [ ]:
# Which country has the highest and lowest average rating?
avg_score = df.loc[:,['country_name', 'aggregate_rating']].groupby('country_name').mean().round(2)
avg_score = avg_score.sort_values(by='aggregate_rating', ascending=False).reset_index()
avg_score

palette = get_country_palette(avg_score['country_name'])

df_plot = avg_score.rename(columns={
        'aggregate_rating': 'Rating (0-5)',
        'country_name': 'Country Name'
    })

fig = px.bar(
    df_plot,
    x='Country Name',
    y='Rating (0-5)',
    title='Average Rating (0-5) by Country',
    color='Country Name',
    text='Rating (0-5)',
    color_discrete_sequence=palette
)
fig.update_layout(showlegend=False)
fig.update_traces(textposition='auto')
fig.show()


In [ ]:
#What is the medium price of a plate for two?
avg_price_for_two = df.loc[:, ['country_name', 'average_cost_dolar']].groupby(['country_name']).agg({'average_cost_dolar': 'mean'}).round(2)
avg_price_for_two = avg_price_for_two.sort_values(by='average_cost_dolar', ascending=False).reset_index()
avg_price_for_two
palette = get_country_palette(avg_price_for_two['country_name'])

df_plot = avg_price_for_two.rename(columns={
        'average_cost_dolar': 'Cost in US Dollars',
        'country_name': 'Country Name'
    })

fig = px.bar(
    df_plot,
    x='Country Name',
    y='Cost in US Dollars',
    title='Average price for two in USD',
    color='Country Name',
    text='Cost in US Dollars',
    color_discrete_sequence=palette
)
fig.update_layout(showlegend=False)
fig.update_traces(textposition='auto')
fig.show()

# Cities analysis

In [ ]:
#Which city has most restaurants?
city_restaurants = df.loc[:, ['city', 'restaurant_id']].groupby('city').count()
city_restaurants = city_restaurants.sort_values(by='restaurant_id', ascending=False).head(10).reset_index()
city_restaurants



In [ ]:
# Top 20 cidades com mais restaurantes, ordenando apenas pela quantidade de restaurantes

city_restaurants = df.groupby(['city', 'country_name'])['restaurant_id'].count().reset_index()
city_restaurants = city_restaurants.sort_values(by='restaurant_id', ascending=False).head(20)

# Pegue os países únicos na ordem de aparição nas 20 cidades
unique_countries = city_restaurants['country_name'].unique()
palette = [COUNTRY_COLORS[country] for country in unique_countries]

df_plot = city_restaurants.rename(columns={
    'city': 'City Name',
    'restaurant_id': 'Restaurants Count',
    'country_name': 'Country Name'
})

# Para garantir a ordem correta das barras (por quantidade de restaurantes)
df_plot['City Name'] = pd.Categorical(df_plot['City Name'], categories=df_plot.sort_values('Restaurants Count', ascending=False)['City Name'], ordered=True)

fig = px.bar(
    df_plot,
    x='City Name',
    y='Restaurants Count',
    color='Country Name',  # cor por país
    title='Top 20 cities with the most number of restaurants',
    color_discrete_sequence=palette,
    category_orders={'City Name': list(df_plot.sort_values('Restaurants Count', ascending=False)['City Name'])}
)
fig.update_layout(showlegend=True)
fig.update_traces(textposition='auto')
fig.show()

In [ ]:
#Which city has more restaurants with rating 4 or better?
city_restaurants_rating = df.loc[df['aggregate_rating'] >= 4, ['city', 'aggregate_rating' ]].groupby('city').count().round(2)
city_restaurants_rating = city_restaurants_rating.sort_values(by='aggregate_rating', ascending=False).reset_index()
city_restaurants_rating

In [ ]:
city_restaurants_rating = df.loc[df['aggregate_rating'] >= 4, ['city', 'country_name', 'aggregate_rating']]
city_restaurants_rating = city_restaurants_rating.groupby(['city', 'country_name']).count().reset_index()
city_restaurants_rating = city_restaurants_rating.sort_values(by='aggregate_rating', ascending=False).head(10)
# Renomeia colunas para o gráfico
df_plot = city_restaurants_rating.rename(columns={
    'city': 'City Name',
    'aggregate_rating': 'Restaurants Count',
    'country_name': 'Country Name'
})

# Garante a ordem das barras por quantidade de restaurantes
df_plot['City Name'] = pd.Categorical(
    df_plot['City Name'],
    categories=df_plot.sort_values('Restaurants Count', ascending=True)['City Name'],
    ordered=True
)

# Paleta de cores dos países presentes no gráfico
unique_countries = df_plot['Country Name'].unique()
palette = [COUNTRY_COLORS[country] for country in unique_countries]

fig = px.bar(
    df_plot,
    x='City Name',
    y='Restaurants Count',
    color='Country Name',
    title='Top 10 cities with the most restaurants rated 4.0/5.0 or above',
    color_discrete_sequence=palette,
    category_orders={'City Name': list(df_plot.sort_values('Restaurants Count', ascending=False)['City Name'])}
)
fig.update_layout(showlegend=True)
fig.update_traces(textposition='auto')

In [ ]:
#Which city has more restaurants with rating 2.5 or below?
city_restaurants_rating_low = df.loc[df['aggregate_rating'] <= 2.5 , ['city', 'aggregate_rating' ]].groupby('city').count().round(2)
city_restaurants_rating_low = city_restaurants_rating_low.sort_values(by='aggregate_rating', ascending=False).reset_index()



,city,aggregate_rating
0,Gangtok,32
1,Ooty,19
2,Brasília,15
3,Rio de Janeiro,13
4,São Paulo,13
5,Manchester,5
6,Clarens,4
7,Ankara,3
8,Hamilton,3
9,Patna,2


In [ ]:
# Calcula o total de restaurantes com nota <= 2.5 por cidade
city_restaurants_rating_low = df.loc[df['aggregate_rating'] <= 2.5, ['city', 'country_name', 'aggregate_rating']]
city_restaurants_rating_low = city_restaurants_rating_low.groupby(['city', 'country_name']).count().reset_index()
city_restaurants_rating_low = city_restaurants_rating_low.sort_values(by='aggregate_rating', ascending=False).head(10)

# Renomeia colunas para o gráfico
df_plot = city_restaurants_rating_low.rename(columns={
    'city': 'City Name',
    'aggregate_rating': 'Restaurants Count',
    'country_name': 'Country Name'
})

# Garante a ordem das barras por quantidade de restaurantes
df_plot['City Name'] = pd.Categorical(
    df_plot['City Name'],
    categories=df_plot.sort_values('Restaurants Count', ascending=False)['City Name'],
    ordered=True
)

# Paleta de cores dos países presentes no gráfico
unique_countries = df_plot['Country Name'].unique()
palette = [COUNTRY_COLORS[country] for country in unique_countries]

fig = px.bar(
    df_plot,
    x='City Name',
    y='Restaurants Count',
    color='Country Name',
    title='Top 10 cities with the most restaurants rated 2.5/5.0 or below',
    color_discrete_sequence=palette,
    category_orders={'City Name': list(df_plot.sort_values('Restaurants Count', ascending=False)['City Name'])}
)
fig.update_layout(showlegend=True)
fig.update_traces(textposition='auto')
fig.show()

In [203]:
#Which city has the highest value for two?
city_avg_price = df.loc[:, ['city', 'average_cost_dolar']].groupby('city').mean().round(2)
city_avg_price = city_avg_price.sort_values(by='average_cost_dolar', ascending=False).reset_index().head(20)
city_avg_price

,city,average_cost_dolar
0,Pasay City,298.96
1,Singapore,141.44
2,New York City,101.50
3,Tagaytay City,99.65
4,Makati City,97.82
5,Taguig City,93.22
6,Miami,92.00
7,Mandaluyong City,91.75
8,Marikina City,89.69
9,Pasig City,84.08


In [ ]:
# Top 20 cidades com maior preço médio para dois, ordenando por cidade e colorindo por país

# Calcula o preço médio para dois por cidade e país
city_avg_price = df.groupby(['city', 'country_name'])['average_cost_dolar'].mean().round(2).reset_index()
city_avg_price = city_avg_price.sort_values(by='average_cost_dolar', ascending=False).head(20)

# Renomeia colunas para o gráfico
df_plot = city_avg_price.rename(columns={
    'city': 'City Name',
    'average_cost_dolar': 'Average Cost for Two (USD)',
    'country_name': 'Country Name'
})

# Garante a ordem das barras por preço médio
df_plot['City Name'] = pd.Categorical(
    df_plot['City Name'],
    categories=df_plot.sort_values('Average Cost for Two (USD)', ascending=False)['City Name'],
    ordered=True
)

# Paleta de cores dos países presentes no gráfico
unique_countries = df_plot['Country Name'].unique()
palette = [COUNTRY_COLORS[country] for country in unique_countries]

fig = px.bar(
    df_plot,
    x='City Name',
    y='Average Cost for Two (USD)',
    color='Country Name',
    title='Top 20 cities with the highest average cost for two (USD)',
    color_discrete_sequence=palette,
    category_orders={'City Name': list(df_plot.sort_values('Average Cost for Two (USD)', ascending=False)['City Name'])}
)
fig.update_layout(showlegend=True)
fig.update_traces(textposition='auto')
fig.show()

In [201]:
# Which city has more different cuisines types ?
cuisines_types = df.loc[:,['cuisines', 'city']].groupby(['city']).nunique().sort_values(by='cuisines', ascending=False).head(20)
cuisines_types

,cuisines
city,
Doha,31
Birmingham,30
Manchester,30
Montreal,30
Houston,29
São Paulo,29
Philadelphia,29
Calgary,28
Portland,28


In [ ]:
# Top 20 cidades com mais tipos de culinária, ordenando por cidade e colorindo por país

# Conta o número de tipos de culinária por cidade
cuisines_types = df.groupby(['city', 'country_name'])['cuisines'].nunique().reset_index()
cuisines_types = cuisines_types.sort_values(by='cuisines', ascending=False).head(20)

# Renomeia colunas para o gráfico
df_plot = cuisines_types.rename(columns={
    'city': 'City Name',
    'cuisines': 'Cuisines Types',
    'country_name': 'Country Name'
})

# Garante a ordem das barras por quantidade de tipos de culinária
df_plot['City Name'] = pd.Categorical(
    df_plot['City Name'],
    categories=df_plot.sort_values('Cuisines Types', ascending=False)['City Name'],
    ordered=True
)

# Paleta de cores dos países presentes no gráfico
unique_countries = df_plot['Country Name'].unique()
palette = [COUNTRY_COLORS[country] for country in unique_countries]

fig = px.bar(
    df_plot,
    x='City Name',
    y='Cuisines Types',
    color='Country Name',
    title='Top 20 cities with the most different cuisine types',
    color_discrete_sequence=palette,
    category_orders={'City Name': list(df_plot.sort_values('Cuisines Types', ascending=False)['City Name'])}
)
fig.update_layout(showlegend=True)
fig.update_traces(textposition='auto')
fig.show()

In [ ]:
# Which city has the most restaurants which accept reservation?
city_reservation = df.loc[df['has_table_booking'] == 1, 'city'].value_counts()
city_reservation

In [ ]:
# Which city has more restaurants that delivers?
city_deliver = df.loc[df['is_delivering_now'] == 1, 'city'].value_counts()
city_deliver

In [ ]:
# Which city has the most restaurants with online delivery?
city_deliver = df.loc[df['has_online_delivery'] == 1, 'city'].value_counts()
city_deliver

# Restaurants view

In [ ]:
# Which restaurant has most reviews?
most_reviews = df.loc[:,['restaurant_name','votes']].sort_values(by='votes',ascending=False).reset_index(drop=True)
most_reviews

In [ ]:
#What is the name of the restaurant with the best rating?
best_restaurant_rating = df.loc[:, ['restaurant_name', 'aggregate_rating','restaurant_id']].sort_values(by=['aggregate_rating'], ascending=False)
best = df.loc[df['aggregate_rating'] >= 4.9, ['restaurant_name','restaurant_id','aggregate_rating']].reset_index(drop=True)
best.sort_values(by=['restaurant_id'], na_position ='last', inplace=True)
best

In [ ]:
#Which restaurant has the most expensive average cost for two people?
expensive_restaurant = df.loc[:, ['restaurant_name', 'average_cost_dolar']].sort_values(by='average_cost_dolar', ascending=False)
expensive_restaurant

In [ ]:
#Restaurant with brazilian food with the worst rating?
brazilian_food_worst = df.loc[df['cuisines'] == 'Brazilian', ['aggregate_rating', 'restaurant_name','restaurant_id']].sort_values(by=['aggregate_rating'])
brazilian_food_worst_aux = brazilian_food_worst.loc[brazilian_food_worst['aggregate_rating'] == 0.0, ['restaurant_name','restaurant_id','aggregate_rating']]
brazilian_food_worst_aux.sort_values(by=['restaurant_id'], na_position='last', inplace=True)
brazilian_food_worst_aux

In [ ]:
#Brazilian restaurant with brazilian food with the best rating?

brazilian_best = df.loc[(df['cuisines']=='Brazilian')&(df['country_name']=='Brazil'), ['restaurant_id','restaurant_name','country_name','cuisines','aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
brazilian_best_aux = brazilian_best.loc[brazilian_best['aggregate_rating']==4.9,['restaurant_id','restaurant_name','country_name','cuisines','aggregate_rating']].sort_values(by='restaurant_id')
brazilian_best_aux

In [218]:
#Does the restaurants which accept online order are the ones with the most reviews?
has_online_delivery = df.loc[df['has_online_delivery'] == 1, ['has_online_delivery', 'votes']].mean()
print(has_online_delivery)

has_no_online_delivery = df.loc[df['has_online_delivery'] == 0, ['has_online_delivery', 'votes']].mean()
print(has_no_online_delivery)

has_online_delivery      1.000000
votes                  838.821664
dtype: float64
has_online_delivery      0.000000
votes                  483.453831
dtype: float64


In [ ]:
# Calcula a média de votos para restaurantes com e sem online delivery
votes_online = df.loc[df['has_online_delivery'] == 1, 'votes'].mean().astype(int)
votes_no_online = df.loc[df['has_online_delivery'] == 0, 'votes'].mean().astype(int)

# Prepara os dados para o gráfico
pie_data = pd.DataFrame({
    'Delivery Type': ['Online Delivery', 'No Online Delivery'],
    'Average Votes': [votes_online, votes_no_online]
})

# Plota o gráfico de pizza mostrando o número de votos
fig = px.pie(
    pie_data,
    names='Delivery Type',
    values='Average Votes',
    title='Average number of votes: Online Delivery vs No Online Delivery',
    
)
fig.update_traces(textinfo='label+value')  # Mostra o nome e o valor absoluto
fig.show()

In [ ]:
#Does the restaurants which accept reservations have a higher average price?
has_reservation = df.loc[df['has_table_booking'] == 1, ['has_table_booking', 'average_cost_dolar']].mean()
no_reservation = df.loc[df['has_table_booking'] == 0, ['has_table_booking', 'average_cost_dolar']].mean()

print(has_reservation)
print(no_reservation)

In [ ]:
#Amercican restaurants with japanese cuisine have higher average cost compared to BBQ restaurants?
american_japanese = df.loc[(df['cuisines'] == 'Japanese') & (df['country_name'] == 'United States of America'), ['restaurant_name', 'country_name', 'cuisines', 'average_cost_dolar']]
american_japanese_mean = american_japanese.groupby(['cuisines', 'country_name'])['average_cost_dolar'].mean()

american_bbq = df.loc[(df['cuisines'] == 'BBQ') & (df['country_name'] == 'United States of America'), ['restaurant_name', 'country_name', 'cuisines', 'average_cost_dolar']]
american_bbq_mean = american_bbq.groupby(['cuisines', 'country_name'])['average_cost_dolar'].mean()

print(american_japanese_mean)
print(american_bbq_mean)

In [ ]:
#Graphic of restaurant price classification
restaurant_price_classification = df.loc[:, ['price_type']].groupby('price_type').value_counts()
restaurant_price_classification

fig = px.pie(restaurant_price_classification.reset_index(), values='count', names='price_type', title='Price Classification', hole=0.3)
fig.update_traces(textinfo='percent+label', # Exibe a porcentagem e o nome da fatia
                  textposition='inside',      # Posiciona o texto dentro da fatia
                  textfont_size=12)   

# Cuisines Types

In [9]:
def best_rated_restaurants_by_cuisine(df, cuisine):
    """
    Retorna restaurantes de uma culinária específica com a maior nota encontrada, ordenados pelo restaurant_id.
    """
    filtered = df.loc[df['cuisines'] == cuisine, ['restaurant_name', 'restaurant_id', 'aggregate_rating']]
    if filtered.empty:
        return filtered  # Retorna vazio se não houver restaurantes dessa culinária
    max_rating = filtered['aggregate_rating'].max()
    best = filtered.loc[filtered['aggregate_rating'] == max_rating]
    return best.sort_values(by='restaurant_id', ascending=True)



In [10]:
def worst_rated_restaurants_by_cuisine(df, cuisine):
    """
    Retorna restaurantes de uma culinária específica com a menor nota encontrada, ordenados pelo restaurant_id.
    """
    filtered = df.loc[df['cuisines'] == cuisine, ['restaurant_name', 'restaurant_id', 'aggregate_rating']]
    if filtered.empty:
        return filtered  # Retorna vazio se não houver restaurantes dessa culinária
    min_rating = filtered['aggregate_rating'].min()
    worst = filtered.loc[filtered['aggregate_rating'] == min_rating]
    return worst.sort_values(by='restaurant_id', ascending=True)



In [ ]:
#Italian food best and worst restaurants
best_rated_restaurants_by_cuisine(df,'Italian')
worst_rated_restaurants_by_cuisine(df, 'Italian')

In [ ]:
#American cuisine best and worst restaurants
best_rated_restaurants_by_cuisine(df, 'American').head(1)
worst_rated_restaurants_by_cuisine(df, 'American')

In [ ]:
#Arabian cuisine best and worst restaurants
best_rated_restaurants_by_cuisine(df, 'Arabian')
worst_rated_restaurants_by_cuisine(df, 'Arabian')

In [ ]:
#Japanese cuisine best and worst restaurants
best_rated_restaurants_by_cuisine(df, 'Japanese')
worst_rated_restaurants_by_cuisine(df, 'Japanese')

In [ ]:
best_rated_restaurants_by_cuisine(df, 'Home-made')
worst_rated_restaurants_by_cuisine(df, 'Home-made')

In [ ]:
# Wchich cuisine has the highest price for two?
cuisine_price = df.loc[:, ['cuisines', 'average_cost_dolar']].groupby('cuisines').mean().sort_values('average_cost_dolar', ascending=False)  
cuisine_price

In [ ]:
# Which cuisie has the best mean rating?
cuisine_rating = df.loc[:, ['cuisines','aggregate_rating']].groupby('cuisines').mean().sort_values(by='aggregate_rating', ascending=False)
cuisine_rating

In [ ]:
# Which cuisanes have the most restaurants which accept online order and delivery?
cuisine_delivery_online = df.loc[(df['has_online_delivery'] == 1) & (df['is_delivering_now'] == 1), ['cuisines']].value_counts()
cuisine_delivery_online

In [ ]:

#Map with all the restaurants marked
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

map_ = folium.Map(location=[center_lat, center_lon], zoom_start=2.5, control_scale=True)
folium.TileLayer(tiles= 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}' ,
                 attr= 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012',
                 name= "Esri.WorldStreetMap").add_to(map_)
marker_cluster = MarkerCluster().add_to(map_)

def add_marker(row):
    popup_html = f"""<b>{row['restaurant_name']}</b><br>
    Rating: {row['aggregate_rating']}/5<br>
    Cuisine: {row['cuisines']}<br>
    Price: {row['price_type']}"""
    iframe = folium.IFrame(html=popup_html, width=250, height=120)
    folium.Marker(
        [row['latitude'], row['longitude']],
        popup=folium.Popup(iframe),
        icon=folium.Icon(icon='cutlery', prefix='glyphicon', icon_color='white', color=row['color_name'])
    ).add_to(marker_cluster)

df.apply(add_marker, axis=1)

map_
